In [14]:
import serial
import numpy as np
import bnkreader
import time

In [15]:
ls

LICENSE              data analysis.ipynb  recording.ipynb
README.md            diagrams/            testfile.bin
__pycache__/         firmware/
bnkreader.py         recording.bin*


In [16]:
#connect to teensy
portname=bnkreader.findteensy()
print(portname)
if (not portname):
         print('no teensy found')
else:
    ser= serial.Serial(portname, 4000000, timeout=10)

/dev/cu.usbmodem7655451


In [17]:
#set Vref voltage units volts
bnkreader.dac_set_voltage(ser,0.7)

In [5]:
#start a recording
framerate= 40000 #frame rate in Hz
recordingtime= 10 #seconds to record for
framechunks= int(framerate*recordingtime/32)
auxchannel= 1 #aux channel
adcrange= 1# adc range, 0 is 0V-2.5V, 1 is 0V-5V
userdata0=101# just any 32 bit int number for user purposes
userdata1=103# just any 32 bit int number for user purposes
bnkreader.recording_start(ser,framerate,framechunks,auxchannel,adcrange,userdata0,userdata1)

b'r40000,12500,1,1,101,103\n'


40000.0

In [6]:
# The LED should turn green if the recording is running
# if it turns blue a frame is skipped because the SD card is slow

In [13]:
# Check status during or after recording
(recording,chunkssaved,framesskipped,data)=bnkreader.status(ser)
print("recording?: %d"%recording)
print("chunks saved: %d"%chunkssaved)
print("frames skipped: %d"%framesskipped)
(framenums,volts,channels,maxvoltage,userdata0s,userdata1s)=bnkreader.decodeframe(data)
print("frame num: %d"%framenums[0])
print("max voltage: %f"%maxvoltage)
print("userdata0: %f"%userdata0s[0])
print("userdata1: %f"%userdata1s[0])
print("Last saved frame(V):")
np.set_printoptions(precision=3)
print(volts[:,:,0])
np.set_printoptions(precision=8)

recording?: 0
chunks saved: 12500
frames skipped: 0
frame num: 399999
max voltage: 4.998779
userdata0: 101.000000
userdata1: 103.000000
Last saved frame(V):
[[4.999 1.964 2.    1.951 1.992 1.952 1.997 1.958 2.017 1.996 1.981 1.94 ]
 [4.999 1.962 1.996 1.951 1.99  1.953 1.989 1.949 2.006 1.962 1.982 1.94 ]
 [4.999 1.962 1.996 1.953 1.992 1.948 1.996 1.954 2.003 1.952 1.981 1.94 ]
 [4.999 1.96  1.99  1.945 1.99  1.951 1.989 1.948 2.    1.951 1.984 1.941]
 [4.999 1.96  1.997 1.948 1.998 1.954 1.992 1.952 1.997 1.949 1.981 1.94 ]
 [4.999 1.959 1.993 1.947 1.991 1.947 1.987 1.952 1.998 1.947 1.981 1.94 ]
 [4.999 1.959 1.993 1.948 1.995 1.952 1.993 1.956 2.001 1.948 1.982 1.94 ]
 [4.999 1.959 1.991 1.948 1.99  1.953 1.989 1.949 1.998 1.948 1.981 1.94 ]
 [4.999 1.959 1.997 1.951 1.993 1.947 1.995 1.953 1.998 1.947 1.984 1.941]
 [1.99  1.948 1.991 1.949 2.    1.948 1.984 1.941 0.021 1.924 1.989 1.942]]


In [ ]:
#save data to a file for latter processing, this may take some time
#wait till recording is done
(recording,chunkssaved,framesskipped,data)=bnkreader.status(ser)
while recording==1:
    time.sleep(1)
    (recording,chunkssaved,framesskipped,data)=bnkreader.status(ser)
with open('testfile.bin','wb') as f:
    for chunkid in range(chunkssaved):
        chunkdata=bnkreader.frame_chunk_readout(ser,chunkid)
        f.write(chunkdata)
        if(chunkid%1000 == 0):
            print('saved chunk %d of %d'%(chunkid+1,chunkssaved))

In [ ]:
#Use this to abort a recording and eject the sd card
#bnkreader.eject_card(ser)

In [ ]:
ser.close()